Задание 1

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [1]:
import pandas as pd

horse_stats_full = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv')
horse_stats_full.columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14 ', '15', '16','17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']
horse_stats_full.head()

,1,2,3,4,5,6,7,8,9,10,...,19,20,21,22,23,24,25,26,27,28
0,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
1,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
2,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
3,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2
4,2,1,528355,?,?,?,2,1,3,1,...,?,?,?,?,1,2,0,0,0,2


In [2]:
df = horse_stats_full[['1','2','4','5','6','7','11','23']]
df.columns = ['surgery?','Age','rectal temperature','pulse','respiratory rate','temperature of extremities','pain','outcome']
df.head(5)

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,1,39.2,88,20,?,3,3
1,2,1,38.30,40,24,1,3,1
2,1,9,39.10,164,84,4,2,2
3,2,1,37.30,104,35,?,?,2
4,2,1,?,?,?,2,2,1


Задание 2

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [3]:
# считаем статистику для всех значений, кроме ?. для этого конвертируем эти значения во float
new_df = df.mask(df == '?').astype(float)

# создаем датафрем со статистикой
stats_new_df = new_df.describe()

# считаем медиану
median = {}
mode = {}

for column in new_df:
    median[column] = new_df[column].median()
    mode[column] = new_df[column].mode()[0]

# считаем медиану добавляем медиану в датафрейм
stats_new_df = pd.concat([stats_new_df, pd.DataFrame([median], index=['median'])])
stats_new_df = pd.concat([stats_new_df, pd.DataFrame([mode], index=['mode'])])

stats_new_df

/Users/kristinaastakhova/Library/Python/3.8/lib/python/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,298.000000,299.00000,239.000000,275.000000,241.000000,243.000000,244.000000,298.000000
mean,1.395973,1.64214,38.166527,71.934545,30.427386,2.345679,2.942623,1.550336
std,0.489881,2.17730,0.733508,28.680522,17.678256,1.046369,1.303993,0.737967
min,1.000000,1.00000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.00000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000
median,1.000000,1.00000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
mode,1.000000,1.00000,38.000000,48.000000,20.000000,3.000000,3.000000,1.000000


In [4]:
# уберем пропуски, чтобы они не отражались на выбросах
new_df = new_df.dropna()

# определим выбросы для пульса по межквартильному расстоянию
def count_quantile(dataframe, column):
    q1 = dataframe[column].quantile(0.25)
    q3 = dataframe[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)
    return lower_bound, upper_bound

lower_bound, upper_bound = count_quantile(new_df, 'pulse')

outliers_pulse = new_df[new_df['pulse'].between(lower_bound, upper_bound, inclusive=True)]
pd.concat([new_df, outliers_pulse]).drop_duplicates(keep=False)

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
2,1.0,9.0,39.1,164.0,84.0,4.0,2.0,2.0
54,1.0,9.0,38.6,160.0,20.0,3.0,3.0,2.0
254,1.0,9.0,38.8,184.0,84.0,1.0,4.0,2.0
274,1.0,9.0,38.8,150.0,50.0,1.0,5.0,2.0


In [5]:
# попробуем взять выбросы для пульса, отталкиваясь от границы нормы пульса 20-40. 120 встречается слишком часто,
# отсеем значения выше 120
new_df = df.mask(df == '?').astype(float)
new_df = new_df.dropna()

print(new_df[(new_df['pulse'] > 120)]['pulse'].value_counts())

124.0    2
140.0    2
150.0    1
184.0    1
129.0    1
136.0    1
130.0    1
160.0    1
164.0    1
Name: pulse, dtype: int64


In [7]:
# определим выбросы для частоты дыхания по межквартильному расстоянию
new_df = df.mask(df == '?').astype(float)
new_df = new_df.dropna()

lower_bound, upper_bound = count_quantile(new_df, 'respiratory rate')

outliers_respiratory_rate = new_df[new_df['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)]
pd.concat([new_df, outliers_respiratory_rate]).drop_duplicates(keep=False)

# для жеребят допустимы показатели до 40, рассчет реалистичен

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
2,1.0,9.0,39.1,164.0,84.0,4.0,2.0,2.0
81,1.0,9.0,38.1,100.0,80.0,3.0,3.0,1.0
83,1.0,1.0,37.8,60.0,80.0,1.0,2.0,1.0
102,1.0,9.0,38.0,140.0,68.0,1.0,3.0,1.0
119,1.0,1.0,39.4,54.0,66.0,1.0,2.0,1.0
124,1.0,1.0,38.0,42.0,68.0,4.0,3.0,1.0
243,1.0,9.0,38.2,124.0,88.0,1.0,2.0,1.0
254,1.0,9.0,38.8,184.0,84.0,1.0,4.0,2.0


Задание 3

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [8]:
# посчитаем пропуски для всех колонок

def count_values(df):
    for column in df: 
        print(df[column].value_counts())
        
count_values(df)

1    180
2    118
?      1
Name: surgery?, dtype: int64
1    275
9     24
Name: Age, dtype: int64
?        60
38.00    16
37.80    13
38.30    12
38.40    10
         ..
39.3      1
37.2      1
37.4      1
37.1      1
39.90     1
Name: rectal temperature, Length: 66, dtype: int64
48     28
60     25
?      24
40     18
88     12
52     12
72     11
100    11
44     11
42     10
120    10
64      8
84      8
80      8
96      8
56      7
54      7
50      6
104     5
66      5
68      5
70      4
76      4
92      4
36      3
90      3
112     3
78      3
86      3
108     3
45      2
150     2
124     2
130     2
30      2
140     2
114     2
160     1
184     1
65      1
146     1
38      1
164     1
98      1
75      1
132     1
110     1
128     1
129     1
136     1
49      1
82      1
46      1
Name: pulse, dtype: int64
?     58
20    28
24    27
16    22
12    19
30    19
40    17
36    16
28    12
32    11
18     8
48     6
60     4
14     4
42     3
68     3
44     3
80     3
1

In [10]:
# Пропуски:
# surgery?: 1 
# Age: 0
# rectal temperature: 60
# pulse: 24
# respiratory rate: 58
# temperature of extremities: 56
# pain - a subjective judgement of the horse's pain level: 55
# outcome: 1

# Удалим строки, где больше трех пропусков
new_df = df.mask(df == '?').astype(float)
new_df = new_df.dropna(thresh=6)

# а также строку, где нет данных по surgery и outcome

new_df = new_df.dropna(subset=['surgery?', 'outcome'], axis=0)

In [11]:
# удалим выбросы для частоты дыхания

lower_bound, upper_bound = count_quantile(new_df, 'respiratory rate')
new_df = new_df[new_df['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)]

# и пульса
new_df = new_df[(new_df['pulse'] < 120)]

new_df

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1.0,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1.0,38.3,40.0,24.0,1.0,3.0,1.0
3,2.0,1.0,37.3,104.0,35.0,NaN,NaN,2.0
5,1.0,1.0,37.9,48.0,16.0,1.0,3.0,1.0
7,2.0,1.0,NaN,80.0,36.0,3.0,4.0,3.0
...,...,...,...,...,...,...,...,...
293,2.0,1.0,38.5,40.0,16.0,1.0,2.0,1.0
295,2.0,1.0,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1.0,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1.0,36.5,100.0,24.0,3.0,3.0,1.0


In [68]:
# 1:  surgery?
#           1 = Yes, it had surgery
#           2 = It was treated without surgery

# 2:  Age
#           1 = Adult horse
#           2 = Young (< 6 months)

# 4:  rectal temperature
#           - linear
#           - in degrees celsius.
#           - An elevated temp may occur due to infection.
#           - temperature may be reduced when the animal is in late shock
#           - normal temp is 37.8
#           - this parameter will usually change as the problem progresses
#                eg. may start out normal, then become elevated because of
#                    the lesion, passing back through the normal range as the
#                    horse goes into shock
# 5:  pulse
#           - linear
#           - the heart rate in beats per minute
#           - is a reflection of the heart condition: 30 -40 is normal for adults
#           - rare to have a lower than normal rate although athletic horses
#             may have a rate of 20-25
#           - animals with painful lesions or suffering from circulatory shock
#             may have an elevated heart rate

# 6:  respiratory rate
#           - linear
#           - normal rate is 8 to 10
#           - usefulness is doubtful due to the great fluctuations

# 7:  temperature of extremities
#           - a subjective indication of peripheral circulation
#           - possible values:
#                1 = Normal
#                2 = Warm
#                3 = Cool
#                4 = Cold
#           - cool to cold extremities indicate possible shock
#           - hot extremities should correlate with an elevated rectal temp.

# 11: pain - a subjective judgement of the horse's pain level
#           - possible values:
#                1 = alert, no pain
#                2 = depressed
#                3 = intermittent mild pain
#                4 = intermittent severe pain
#                5 = continuous severe pain
#           - should NOT be treated as a ordered or discrete variable!
#           - In general, the more painful, the more likely it is to require
#             surgery
#           - prior treatment of pain may mask the pain level to some extent
        
# 23: outcome
#           - what eventually happened to the horse?
#           - possible values:
#                1 = lived
#                2 = died
#                3 = was euthanized

In [12]:
# рассчитаем медианные и средние показатели в зависимости от возраста лошади

print(new_df.groupby('Age').median().round())
print(new_df.groupby('Age').mean().round())

     surgery?  rectal temperature  pulse  respiratory rate  \
Age                                                          
1.0       1.0                38.0   60.0              24.0   
9.0       2.0                38.0   88.0              30.0   

     temperature of extremities  pain  outcome  
Age                                             
1.0                         3.0   3.0      1.0  
9.0                         2.0   3.0      1.0  
     surgery?  rectal temperature  pulse  respiratory rate  \
Age                                                          
1.0       1.0                38.0   64.0              26.0   
9.0       2.0                39.0   90.0              34.0   

     temperature of extremities  pain  outcome  
Age                                             
1.0                         2.0   3.0      2.0  
9.0                         2.0   3.0      1.0  


In [13]:
# посмотрим, есть ли корреляция с итогом

print(new_df.groupby('outcome').median().round())
print(new_df.groupby('outcome').mean().round())

         surgery?  Age  rectal temperature  pulse  respiratory rate  \
outcome                                                               
1.0           2.0  1.0                38.0   52.0              24.0   
2.0           1.0  1.0                38.0   78.0              30.0   
3.0           1.0  1.0                38.0   78.0              24.0   

         temperature of extremities  pain  
outcome                                    
1.0                             2.0   3.0  
2.0                             3.0   4.0  
3.0                             3.0   4.0  
         surgery?  Age  rectal temperature  pulse  respiratory rate  \
outcome                                                               
1.0           2.0  1.0                38.0   59.0              24.0   
2.0           1.0  1.0                38.0   74.0              30.0   
3.0           1.0  1.0                38.0   75.0              28.0   

         temperature of extremities  pain  
outcome                 

In [14]:
# посмотрим, есть ли корреляция с болью

print(new_df.groupby('pain').median().round())
print(new_df.groupby('pain').mean().round())

      surgery?  Age  rectal temperature  pulse  respiratory rate  \
pain                                                               
1.0        2.0  1.0                38.0   48.0              20.0   
2.0        2.0  1.0                38.0   70.0              24.0   
3.0        1.0  1.0                38.0   60.0              24.0   
4.0        1.0  1.0                38.0   72.0              30.0   
5.0        1.0  1.0                38.0   83.0              30.0   

      temperature of extremities  outcome  
pain                                       
1.0                          2.0      1.0  
2.0                          3.0      1.0  
3.0                          2.0      1.0  
4.0                          3.0      2.0  
5.0                          3.0      2.0  
      surgery?  Age  rectal temperature  pulse  respiratory rate  \
pain                                                               
1.0        2.0  1.0                38.0   50.0              21.0   
2.0        

In [15]:
# посмотрим, есть ли корреляция с температурой конечностей

print(new_df.groupby('temperature of extremities').median().round())
print(new_df.groupby('temperature of extremities').mean().round())

                            surgery?  Age  rectal temperature  pulse  \
temperature of extremities                                             
1.0                              1.0  1.0                38.0   52.0   
2.0                              2.0  1.0                38.0   54.0   
3.0                              1.0  1.0                38.0   69.0   
4.0                              1.0  1.0                38.0   80.0   

                            respiratory rate  pain  outcome  
temperature of extremities                                   
1.0                                     24.0   3.0      1.0  
2.0                                     30.0   3.0      1.0  
3.0                                     26.0   3.0      1.0  
4.0                                     30.0   3.0      1.0  
                            surgery?  Age  rectal temperature  pulse  \
temperature of extremities                                             
1.0                              1.0  1.0          

In [16]:
# заполняем пропуски

#     Выводы по взаимосвязям:
# молодые лошади чаще умирают
# взрослым лошадям чаще делают операцию
# у взрослых лошадей выше частота дыхания
# лошади, которым не делали операцию, чаще выживают
# у лошадей, которые не выжили, выше пульс и боль
# у лошадей, которые умерли, выше частота дыхания
# лошади, которым не делали операцию, испытывают меньше боли
# чем выше пульс, тем больше боль и частота дыхания
# чем выше температура конечностей, тем выше пульс и частота дыхания
# лошади, у которых выше температура конечностей, не выживали

# для ректальной температуры поставим среднее
new_df['rectal temperature'].fillna(new_df['rectal temperature'].mean().round(), inplace=True)

# для боли — зависимость от исхода и операции
new_df['pain'].fillna(new_df.groupby(['surgery?', 'outcome'])['pain'].transform('median'), inplace=True)

# для частоты дыхания — зависимость от возраста, исхода, температуры конечностей, пульса
new_df['respiratory rate'].fillna(new_df.groupby(['Age', 'outcome', 'temperature of extremities', 'pulse'])['respiratory rate'].transform('median'), inplace=True)

# для температуры конечностей — зависимость от исхода и частоты дыхания
new_df['temperature of extremities'].fillna(new_df.groupby(['outcome', 'respiratory rate'])['temperature of extremities'].transform('median'), inplace=True)

In [17]:
new_df

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1.0,39.2,88.0,20.0,3.0,3.0,3.0
1,2.0,1.0,38.3,40.0,24.0,1.0,3.0,1.0
3,2.0,1.0,37.3,104.0,35.0,3.0,3.0,2.0
5,1.0,1.0,37.9,48.0,16.0,1.0,3.0,1.0
7,2.0,1.0,38.0,80.0,36.0,3.0,4.0,3.0
...,...,...,...,...,...,...,...,...
293,2.0,1.0,38.5,40.0,16.0,1.0,2.0,1.0
295,2.0,1.0,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1.0,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1.0,36.5,100.0,24.0,3.0,3.0,1.0


In [18]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 298
Data columns (total 8 columns):
surgery?                      201 non-null float64
Age                           201 non-null float64
rectal temperature            201 non-null float64
pulse                         201 non-null float64
respiratory rate              201 non-null float64
temperature of extremities    197 non-null float64
pain                          201 non-null float64
outcome                       201 non-null float64
dtypes: float64(8)
memory usage: 14.1 KB


In [19]:
# P.S.: не понимаю, почему не заполнились 4 значения для температуры конечностей, и просто их удалю
new_df = new_df.dropna(subset=['temperature of extremities'], axis=0)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197 entries, 0 to 298
Data columns (total 8 columns):
surgery?                      197 non-null float64
Age                           197 non-null float64
rectal temperature            197 non-null float64
pulse                         197 non-null float64
respiratory rate              197 non-null float64
temperature of extremities    197 non-null float64
pain                          197 non-null float64
outcome                       197 non-null float64
dtypes: float64(8)
memory usage: 13.9 KB
